## Introduction

Real data is messy. If it's because of changes in standards, human error, or just because the real data points are messy, when taking real data from insitutions that were usually not keeping out around for the purposes of data science, we tend to encounter inconsistent formatting, missing values, duplicates, inconsistent typing, and other issues.

`dftest` (inspired by `pytest`) is a project aims to give data scientists tools to detect problematic data which may lead to unexpected results, and loctae these rows and columns which may need to be removed or require additional cleaning.

## Installation & Setup

In [ ]:
# !pip3 install --index-url https://test.pypi.org/simple/ --upgrade  dftestbash

%%bash
builddir=$(mktemp -d)
git clone https://github.com/TheAllSeeing/PandasTestingSuite $builddir
pip3 install $builddir

Processing /tmp/tmp.789Q3dhBrV
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for dftest: filename=dftest-0.2.1-py3-none-any.whl size=18667 sha256=bb823dc348bc851e452b3ad99a2219c7a75007371af21952d2c48435e098a86d
  Stored in directory: /tmp/pip-ephem-wheel-cache-57y9cwgc/wheels/10/27/81/71304e43a272489a26c30a53433bab66afb802486e75eb63f9
Successfully built dftest
  Attempting uninstall: dftest
    Found existing installation: dftest 0.2.1
    Uninstalling dftest-0.2.1:
      Successfully uninstalled dftest-0.2.1


Cloning into '/tmp/tmp.789Q3dhBrV'...
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
# Packages we'll need for this demonstration
import pandas as pd # Required for working with dftest
import numpy as np # Will be used for randomly populating dataframes

from dftest import DFTests, tests

## Demonstration 1: ensuring positive numbers

Let's look at how this tool can be used with some very basic data frames.

### Step 0: get a dataframe to test


Of course, in order to test the integrity of _any_ data, we first need to have it.

For this minimal example, we're create our own random dataset with 4 columns of integers between 0 and 100 and a fifth column containing lower case letters.

In [ ]:
df =  pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list('ABCD'))
df['E'] = np.random.choice([chr(i) for i in range(97, 121)], size=100)
df.to_csv('example.csv', index_label='Index')

In [ ]:
pd.read_csv('example.csv')

,Index,A,B,C,D,E
0,0,51,90,75,55,k
1,1,60,39,8,44,n
2,2,42,33,1,59,d
3,3,12,89,15,47,j
4,4,25,85,42,62,f
...,...,...,...,...,...,...
95,95,18,80,18,83,c
96,96,7,84,69,19,p
97,97,55,20,16,40,u
98,98,11,99,72,28,d


### Step 1: think about something to test

For this example, let's say we'd like to make sure all the values in column B are positive.

#### Do we want a _boolean_ or _index_ test?
<a name="test-types"></a>
`dftest` tests can be any function that takes in a dataframe and output either:
- A boolean value indicating column validity ("Boolean Test").
- A list of hashable values (usually ints) indicating indexes of invalid rows ("Index Test").

As a rule of thumb, if you collect more information (i.e return row indexes instead of just true/false value), you will get more extensive analytics.
 
We might not care much for extensive analytics though, and just want to ensure a large enough portion of the column is valid. So for this example, we'll define a **boolean test** which ensures at least 95% of the column is positive.

### Step 2: define your test

Test functions must start with the prefix `dftest` in order to be recognized. the package supports snake_case, PascalCase and camelCase (`DFTestCheck`, `dftest_check`, `df_test_check`, `DfTestCheck`, `dfTestCheck`, etc.)

Excpeting the stated constraints on parameters and return value, a test function can be any python function, regardless of where or how it's defined.

```python
# Option 1: lambda
dftest_b_positive = lambda df: sum(1 for cell in dataframe['B'] if cell > 0) / len(df.index) >= 0.95

# Option 2: def syntax
def dftest_b_positive(dataframe: pd.DataFrame):
  valid_count = sum(1 for cell in dataframe['B'] if cell > 0)
  row_count = len(dataframe.index)
  res = valid_count / row_count >= 0.95
  return res

# Option 3: whatever you feel like!
```

### Step 3: save the test in a file

Much like `pytest`, the `dftest` command runs on python files containing test functions (though unlike `pytest`, they operate on some dataset that can be external to the code). 

In [ ]:
%%writefile tests.py
def dftest_b_positive(dataframe):
  valid_count = dataframe['B'].apply(lambda x: x > 0).sum()
  row_count = len(dataframe.index)
  res = valid_count / row_count >= 0.95
  return res

Overwriting tests.py


### Step 4: running the tests:

In the most basic usage of the `dftest` command, we specify a dataset file (in CSV, TSV, Excel or JSON formats) and a list of files containing test functions. Note that each 

In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_b_positive)
Finished testing
Columns Tested: 1/6 (17%).
Columns valid: 1/1 (100.0%).



### Step 5: looking at dataframe results

You can add certain flags to print more information or show analytics.

- `--print` or `-p` prints a summary of coverage and details results for individual tests in invalid columns. Runs by default if no graph options are specified.

- `--graph` or `-g` opens pyplot graphs according to its arguments
    - `validity` will create and return a pyplot figure with a 1D heatmap of 
    the columns by validity. 
    - `summary` will create and return a pyplot figure with pie charts showing 
    the amount of columns tested and amount of columns valid.
    - `coverage` will create and return a binary-colored heatmap showing which 
    columns were tested.



By running the commands below you can generate a heatmap of the validity for each column. You should see a green bar for column B and 3 white bars for columns A, B and C, marking that they are untested ("lack coverage"). 

The final bar represents the Dataframe, and should be marked valid as well. 

In general, it would be marked invalid for any failed test, though it is unaffected by uncovered columns.

In [ ]:
!dftest --dataframe example.csv --files tests.py --print --graph validity summary

Testing 00% (#1: dftest_b_positive)
Finished testing
Columns Tested: 1/6 (17%).
Columns valid: 1/1 (100.0%).

<Figure size 640x480 with 2 Axes>
<Figure size 640x480 with 1 Axes>


___A small note:___ graphing methods create and return pyplot figures, which causes Colab to display the graphs twice, once when plotted and another one when it returns. To avoid this you can simply put the returned figure into a variable, as is done in this notebook.

In a traditional run, the graphs would not be displayed until pyplot.show() is called. For this purpose, the results class contains a pointer to the pyplot module, such that you can call `results.plt.show()`.

### Step 6: looking at column results

You can show results for a specific column using the `--column` option (short version `-c`). Column results provide a slightly different collection of graphs - 

- There is no `coverage` graph
- `tests` graph displays bar charts comparing sucess of different Index tests. 
- `validity` graph displays a heat map of tests success.
- `summary` only displays a pie chart of test success rate.

In [ ]:
!dftest --dataframe example.csv --files tests.py --column B --print --graph validity summary

Testing 00% (#1: dftest_b_positive)
Finished testing
--- B ---
Test #01: dftest_b_positive: Success

<Figure size 640x480 with 1 Axes>
<Figure size 640x480 with 1 Axes>


### Step 7: Saving results to file


Of course, we don't want to run the tests again every time we want to look at the results. For this reason, you can use the `--dump` option to specify a file to save the results to.

In [ ]:
!dftest --dataframe example.csv --files tests.py --dump results.bin

Testing 00% (#1: dftest_b_positive)
Finished testing
Columns Tested: 1/6 (17%).
Columns valid: 1/1 (100.0%).



Then you can load the results using the `--results` option, omitting the `--dataframe` and `--files` parameters.

In [ ]:
!dftest  --results results.bin --print

Traceback (most recent call last):
  File "/usr/local/bin/dftest", line 68, in <module>
    results = dill.load(dumpfile)
  File "/usr/local/lib/python3.7/dist-packages/dill/_dill.py", line 313, in load
    return Unpickler(file, ignore=ignore, **kwds).load()
  File "/usr/local/lib/python3.7/dist-packages/dill/_dill.py", line 525, in load
    obj = StockUnpickler.load(self)
EOFError: Ran out of input


In [ ]:
!dftest --results results.bin --column B --graph validity

Traceback (most recent call last):
  File "/usr/local/bin/dftest", line 68, in <module>
    results = dill.load(dumpfile)
  File "/usr/local/lib/python3.7/dist-packages/dill/_dill.py", line 313, in load
    return Unpickler(file, ignore=ignore, **kwds).load()
  File "/usr/local/lib/python3.7/dist-packages/dill/_dill.py", line 525, in load
    obj = StockUnpickler.load(self)
EOFError: Ran out of input


**important:** results are saved in the [pickle](https://docs.python.org/3/library/pickle.html#module-pickle) format. It is possible to constrct a maliciouys pickle file which will execute arbitrary code when loaded; _only load files you trust._

### Step 8: seeing the test is working

Let's put some invalid values in B, just to see that our test is really detecting invalid rows. 

In [ ]:
df = pd.read_csv('example.csv', index_col=0)
df.loc[:5, 'B'] *= -1
df.to_csv('example.csv')

In [ ]:
!dftest --dataframe example.csv --files tests.py  --print --graph validity

Testing 00% (#1: dftest_b_positive)
Finished testing
Columns Tested: 1/6 (17%).
Columns valid: 0/1 (0.0%).

--- Column 3: B ---
Test #01: dftest_b_positive: Failure

<Figure size 640x480 with 1 Axes>


Seems to work well enough. Notice how info about the tests of invalid columns are printed after the summary.

_What happens if we put in values that the test is not meant to check,_ like missing values or incorrect types? The short answer: it's up to you. More info can be found in [Demonstration 4](#demonstration-4) and [Demonstration 6](#demonstration-6).

In [ ]:
!which dftest

/usr/local/bin/dftest


## Demonstration 2: generalizing tests

### Step 1: Which columns is this test applicable to?

The test we made is applicable to more than column B. All of our numeric columns here are supposed to be positive. 

However, since we each function can only return a single True/False value, we would need to create different test functions for each of the columns we would want to get results for.  How might we 

### Step 2: Understanding Generic Tests

As a solution to this problem, `dftests` supports tests that rather than just run on a dataframe, runs on __some column__ in a dataframe.


For example:

```python
def dftest_positive(column: str, dataframe: pd.DataFrame):
  valid_count = dataframe[column].apply(lambda x: x > 0).sum()
  row_count = len(dataframe.index)
  res = valid_count / row_count >= 0.95
  return res
```

Rather than make a test that runs on a dataframe, we made a test that runs on a __some column__ and a dataframe.

In this library, we call tests that take in _some column_ as "generic" tests, and they are automatically recognized by the `dftest` command.

By default, when dftest runs into a generic test, it applies it to each column in the dataframe.

***Important:*** _the order of parmeters is significant. trying to add a function that takes (dataframe, column) will cause error and unexpected behaviour_

### Step 3: Tests options & Understanding how to test specific columns

As a general rule, you can configure the way `dftest` treats a test funciton with the `dftest.options` decorator.

For example, the test delinated above should not run on the string column E. We can achieve this in a numer of ways using the `dftest.options` decorator:

```python
# Specifying columns for the test to run on

# by hand
@dftest.options(include=['A', 'B', 'C', 'D'])
def dftest_positive(column: str, dataframe: pd.DataFrame):
  valid_count = sum(1 for cell in dataframe[column] if cell > 0)
  row_count = len(dataframe.index)
  res = valid_count / row_count >= 0.95
  return res

# by dtype
@dftest.options(include_dtypes=[int])
def dftest_positive(column: str, dataframe: pd.DataFrame):
  valid_count = sum(1 for cell in dataframe[column] if cell > 0)
  row_count = len(dataframe.index)
  res = valid_count / row_count >= 0.95
  return res
```
```python
# Specifying columns for the test not to run on
@dftest.options(exclude=['E'])
def dftest_positive(column: str, dataframe: pd.DataFrame):
  valid_count = sum(1 for cell in dataframe[column] if cell > 0)
  row_count = len(dataframe.index)
  res = valid_count / row_count >= 0.95
  return res

```


Decorators only work right above a function definition using `def`. Since you may want to import tests, use lambda exressions and so on, a function `declare_options` is also provided, which you can run on any callable attribute to define its optons.

For example, the following test will check `int` and `float` columns for numbers bigger thana 1,000,000:

```python
import dftests

dftest_big = lambda col, df: df[col].apply(lambda x: x > 1_000_00)
dftests.declare_options(dftest_int, include_dtypes=[int, float)
```

### Step 4: putting it together:

In [ ]:
%%writefile tests.py
import dftest
from pandas import DataFrame # for the type hint

@dftest.options(include_dtypes=[int])
def dftest_positive(column: str, dataframe: DataFrame):
  valid_count = sum(1 for cell in dataframe[column] if cell > 0)
  row_count = len(dataframe.index)
  res = valid_count / row_count >= 0.95
  return res

Overwriting tests.py


Notice we are overwriting the test file rather than appending it, since we want to replace the previous test.

### Step 5: seeing the results:

In [ ]:
!dftest --dataframe example.csv --files tests.py --print --graph validity

Testing 00% (#1: dftest_positive — A)
Testing 20% (#2: dftest_positive — Index)
Testing 40% (#3: dftest_positive — D)
Testing 60% (#4: dftest_positive — C)
Testing 80% (#5: dftest_positive — B)
Finished testing
Columns Tested: 5/6 (83%).
Columns valid: 4/5 (80.0%).

--- Column 3: B ---
Test #01: dftest_positive — B: Failure

<Figure size 640x480 with 1 Axes>


## Demonstration 3: Index Tests

### Step 1: create an index test and add it

You might be interested in getting more information out of your tests. As mentioned, one way to do this is to use _Index Tests_, which return an index of valid and invalid values.

Index Tests may return either
1. An iterable containing indexes for invalid rows.
1. An array or series with boolean values for each series

```python
# Option 1
def dftest_positive(col, df):
  return [i for i, cell in enumerate(df[col]) if cell > 0]

# Option 2
def dftest_positive(col, df):
  return df[col].apply(lambda x: x > 0)
```

In [ ]:
%%writefile tests.py
import dftest

@dftest.options(include_dtypes=[int])
def dftest_positive(col, df):
  return df[col].apply(lambda x: x > 0)

Overwriting tests.py


### Step 2: Get the results

In [ ]:
!dftest --dataframe example.csv --files tests.py --print --graph validity

Testing 00% (#1: dftest_positive — D)
Testing 20% (#2: dftest_positive — B)
Testing 40% (#3: dftest_positive — A)
Testing 60% (#4: dftest_positive — C)
Testing 80% (#5: dftest_positive — Index)
Finished testing
Columns Tested: 5/6 (83%).
Columns valid: 2/5 (40.0%).

--- Column 1: Index ---
Test #01: dftest_positive — Index: 99/100 (99.0%).
   Index
0      0

--- Column 2: A ---
Test #01: dftest_positive — A: 99/100 (99.0%).
    Index  A
21     21  0

--- Column 3: B ---
Test #01: dftest_positive — B: 94/100 (94.0%).
   Index   B
0      0 -90
1      1 -39
2      2 -33
3      3 -89
4      4 -85
5      5 -43

<Figure size 640x480 with 2 Axes>


### Step 3: understanding the graphs

Notice the different shades and the numbers of the columns in the heatmap, instead of the previous clear green bar, as well as the details for each invalid row in the printing.

If no boolean tests were run, the heatmap generated will be more detailed and place columns on a gradient instead of a binary success-fail.



**Error sparsity**

The final bar in the graph represents the dataframe. You can compare it to the individual columns to get a sense of the overlap between failed test; the more overlap, the closer it will be to the actual column values; the less, the lower.

**Getting binary graph back.**

It's possible to get the binary heatmap back by specifying validity-binary as the graph parameter. In this case, success of a row will be determined by the portion of its rows that are valid - by default requiring all of them, but this can be specified when adding the test.

In [ ]:
# Any columns that has at least one nonpositive is red
!dftest --dataframe example.csv --files tests.py --print --graph validity-binary

Testing 00% (#1: dftest_positive — D)
Testing 20% (#2: dftest_positive — C)
Testing 40% (#3: dftest_positive — Index)
Testing 60% (#4: dftest_positive — A)
Testing 80% (#5: dftest_positive — B)
Finished testing
Columns Tested: 5/6 (83%).
Columns valid: 2/5 (40.0%).

--- Column 1: Index ---
Test #01: dftest_positive — Index: 99/100 (99.0%).
   Index
0      0

--- Column 2: A ---
Test #01: dftest_positive — A: 99/100 (99.0%).
    Index  A
21     21  0

--- Column 3: B ---
Test #01: dftest_positive — B: 94/100 (94.0%).
   Index   B
0      0 -90
1      1 -39
2      2 -33
3      3 -89
4      4 -85
5      5 -43

<Figure size 640x480 with 1 Axes>


In [ ]:
%%writefile tests.py
import dftest

@dftest.options(include_dtypes=[int], success_threshold=0.95)
def dftest_positive(col, df):
  return df[col].apply(lambda x: x > 0)

Overwriting tests.py


In [ ]:
# Any columns that has at least 5% nonpositives is red
!dftest --dataframe example.csv --files tests.py --print --graph validity-binary

Testing 00% (#1: dftest_positive — B)
Testing 20% (#2: dftest_positive — A)
Testing 40% (#3: dftest_positive — Index)
Testing 60% (#4: dftest_positive — D)
Testing 80% (#5: dftest_positive — C)
Finished testing
Columns Tested: 5/6 (83%).
Columns valid: 4/5 (80.0%).

--- Column 3: B ---
Test #01: dftest_positive — B: 94/100 (94.0%).
   Index   B
0      0 -90
1      1 -39
2      2 -33
3      3 -89
4      4 -85
5      5 -43

<Figure size 640x480 with 1 Axes>


You can force the non-binary graph even if you have both boolean and index tests. In this case, only index tests will be charted.

In [ ]:
%%writefile tests.py
import dftest

@dftest.options(include_dtypes=[int], success_threshold=0.95)
def dftest_positive(col, df):
  return df[col].apply(lambda x: x > 0)

@dftest.options(include_dtypes=[object])
def dftest_before_n(col, df):
  return sum(1 for cell in df[col] if ord(cell) < 110)

Overwriting tests.py


In [ ]:
# Only the positive test will show up in the graph
!dftest --dataframe example.csv --files tests.py --print --graph validity-nonbinary

Testing 00% (#1: dftest_before_n — E)
Testing 17% (#2: dftest_positive — B)
Testing 33% (#3: dftest_positive — Index)
Testing 50% (#4: dftest_positive — D)
Testing 67% (#5: dftest_positive — A)
Testing 83% (#6: dftest_positive — C)
Finished testing
Columns Tested: 6/6 (100%).
Columns valid: 5/6 (83.33%).

--- Column 3: B ---
Test #01: dftest_positive — B: 94/100 (94.0%).
   Index   B
0      0 -90
1      1 -39
2      2 -33
3      3 -89
4      4 -85
5      5 -43

<Figure size 640x480 with 1 Axes>


### ּStep 4: playing with invalid rows

<a name="demonstration-4"></a>
## Demonstration 4: Coverage 

### Step 0: about coverage

The intended us eof this tool can be analogous to traditional unit testing. In the same way that it's useful not only to see which parts of the code produce unexpected results, but also which parts of the code are not running when they should (or vice versa), it is useful to us to see both which parts of our data are invalid, as well as which parts of the data are not even being tested. 

Below is a minimal scenario denmonstarting this possible usefulness.

As a sidenote, in unit testing the term for checking which parts are being tested vs. untested is _coverage_, and it's the same term I'll use her.

### Step 1: Adding nulls

First, let's put some null values into the dataframe and see what our tests give us.

In [ ]:
df = pd.read_csv('example.csv')
df.loc[:5, 'C'] = None
df.to_csv('example.csv', index=False)

### Step 2: Rerunning the tests

In [ ]:
!dftest --dataframe example.csv --files tests.py --print --graph validity

Testing 00% (#1: dftest_before_n — E)
Testing 20% (#2: dftest_positive — B)
Testing 40% (#3: dftest_positive — D)
Testing 60% (#4: dftest_positive — A)
Testing 80% (#5: dftest_positive — Index)
Finished testing
Columns Tested: 5/6 (83%).
Columns valid: 4/5 (80.0%).

--- Column 3: B ---
Test #01: dftest_positive — B: 94/100 (94.0%).
   Index   B
0      0 -90
1      1 -39
2      2 -33
3      3 -89
4      4 -85
5      5 -43

<Figure size 640x480 with 2 Axes>


You should see that instead of showing C as invalid, it shows it lacking coverage. Why is that?


### Step 3: Investigate


An obvious step is to look at the column's values:

In [ ]:
df['C']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
95    18.0
96    69.0
97    16.0
98    72.0
99    69.0
Name: C, Length: 100, dtype: float64

It seems that pandas converted the column into float, to allow for Nan types. Thus, `df.select_dtypes(int)` does not detect it, and it is not detected.

This is a pretty trivial thing to notice, but we might not have had we not seen the column's lack of coverage, and instead gotten tangled in our own code if this ever came to bite us.

<a name="demonstration-6"></a>
## Demonstration 5: handling Exceptions

### Step 1: Will this cause an exception?

In demonstration 4, the invalid values caused pandas to generalize the type of the column, which hid it from our tests. What would have happened if we picked our columns manually, instead of by type (hence forcing the test over our invalid cells)?

In [ ]:
%%writefile tests.py
import dftest

@dftest.options(exclude=['E'])
def dftest_positive(col, df):
  return df[col].apply(lambda x: x > 0)

Overwriting tests.py


In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_positive — B)
Testing 20% (#2: dftest_positive — D)
Testing 40% (#3: dftest_positive — C)
Testing 60% (#4: dftest_positive — Index)
Testing 80% (#5: dftest_positive — A)
Finished testing
Columns Tested: 5/6 (83%).
Columns valid: 1/5 (20.0%).

--- Column 1: Index ---
Test #01: dftest_positive — Index: 99/100 (99.0%).
   Index
0      0

--- Column 2: A ---
Test #01: dftest_positive — A: 99/100 (99.0%).
    Index  A
21     21  0

--- Column 3: B ---
Test #01: dftest_positive — B: 94/100 (94.0%).
   Index   B
0      0 -90
1      1 -39
2      2 -33
3      3 -89
4      4 -85
5      5 -43

--- Column 4: C ---
Test #01: dftest_positive — C: 94/100 (94.0%).
   Index   C
0      0 NaN
1      1 NaN
2      2 NaN
3      3 NaN
4      4 NaN
5      5 NaN



### Step 2: Will _this_ cause an exception?

Notably, the test runs pretty smoothly. This is because pandas uses NaNs to represent missing values, rather than Nones, allowing traditional numerical tests to run on them (always returning `False`).

What would happen if we tried to put a string in, though?

In [ ]:
df = pd.read_csv('example.csv')
df.loc[:5, 'A'] = 'invalid'
df.to_csv('example.csv', index=False)

In [ ]:
df

,Index,A,B,C,D,E
0,0,invalid,-90,NaN,55,k
1,1,invalid,-39,NaN,44,n
2,2,invalid,-33,NaN,59,d
3,3,invalid,-89,NaN,47,j
4,4,invalid,-85,NaN,62,f
...,...,...,...,...,...,...
95,95,18,80,18.0,83,c
96,96,7,84,69.0,19,p
97,97,55,20,16.0,40,u
98,98,11,99,72.0,28,d


In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_positive — Index)
Testing 20% (#2: dftest_positive — C)
Testing 40% (#3: dftest_positive — D)
Testing 60% (#4: dftest_positive — A)
Traceback (most recent call last):
  File "/usr/local/bin/dftest", line 95, in <module>
    results = dftests.run()
  File "/usr/local/lib/python3.7/dist-packages/dftest/DFTests.py", line 294, in run
    results.append(test.run(self.dataframe))
  File "/usr/local/lib/python3.7/dist-packages/dftest/Test.py", line 107, in run
    result, columns_tested = self.test(dataframe)
  File "/usr/local/lib/python3.7/dist-packages/dftest/Test.py", line 74, in test
    return self.predicate(test_target, **self.kwargs), self.tested_columns.difference(self.ignore_columns)
  File "tests.py", line 5, in dftest_positive
    return df[col].apply(lambda x: x > 0)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/series.py", line 4357, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/usr/local/lib/python

As one might expect, an exception is thrown.


### Step 3: Handling exceptions

`dftest` does not make any assumptoins on the desired behaviour in edge cases, so it is up to the user to catch any exceptions their test might raise. 

Below is a simple example which will flag any non-number values as invalid.

In [ ]:
%%writefile tests.py
import dftest

@dftest.options(exclude=['E'])
def dftest_positive(column, dataframe):
  return dataframe[column] \
  .apply(lambda x: x if isinstance(x, int) or isinstance(x, float) else -1) \
  .apply(lambda x: x > 0)

Overwriting tests.py


In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_positive — Index)
Testing 20% (#2: dftest_positive — A)
Testing 40% (#3: dftest_positive — C)
Testing 60% (#4: dftest_positive — D)
Testing 80% (#5: dftest_positive — B)
Finished testing
Columns Tested: 5/6 (83%).
Columns valid: 1/5 (20.0%).

--- Column 1: Index ---
Test #01: dftest_positive — Index: 99/100 (99.0%).
   Index
0      0

--- Column 2: A ---
Test #01: dftest_positive — A: 0/100 (0.0%).
   Index        A
0      0  invalid
1      1  invalid
2      2  invalid
3      3  invalid
4      4  invalid
5      5  invalid
6      6       80
7      7       17
8      8       26
9      9       47
...

--- Column 3: B ---
Test #01: dftest_positive — B: 94/100 (94.0%).
   Index   B
0      0 -90
1      1 -39
2      2 -33
3      3 -89
4      4 -85
5      5 -43

--- Column 4: C ---
Test #01: dftest_positive — C: 94/100 (94.0%).
   Index   C
0      0 NaN
1      1 NaN
2      2 NaN
3      3 NaN
4      4 NaN
5      5 NaN




## Demonstration 6: the `tests` submodule

### Step 0: What is the `tests` submodule?

`dftests.tests` is a submodule contaning many useful functions to generate common tests, like regex comparisons, numeric range checks and many more. They help remove a lot of boilerplate code and can significantly simplify the process of creating tests.

In addition to the test "makers", there are also a number of normal tests. 

Currently, all of the normal tests are generic Index test.

### Step 1: A new dataframe

For this demonstration, let's look at the following dataframe

In [ ]:
df = pd.DataFrame({
    'A': [1    , 2   , True , 3    , 4      , 'a'  ], # Integers
    'B': ['Yes', 'No', 'Yes', 'No' , 'Maybe', 'No' ], # Yes/No boolean 
    'C': [20   , 45  , 100  , None , 20     , 0    ], # Percentages
    'D': [55   , -1  , 100  , 60   , 120    , 76   ], # Percentages
    'E': [0.243, 0.2 , 0.33 , 1.2  , 0.1    , 0    ], # 0-1 fractions
    'F': ['66.17.230.185', '238.122.190.60', 'ERROR', '129.167.214.X', '217.110.194.132', '106.121.183.215'] # IP addresses
}) 
df.to_csv('example.csv')

### Step 2: How to configure tests?

Unlike previous demonstratoins, we will not be defining the functions in place; rather, generating them roughly like so:

```python
dftest_mytest = tests.make_test(options)
```
Or alternatively, straight importing them:
```python
from dftest.tests import dftest_positive
```

The `dftest` command simply searches for callable attributes with the right prefix, so it will detect both of these, even the import.

Notably though, we cannot use the `options` decorator to configure our tests, as they are not defined using `def`. Instead. the function `dftests.declare_options` may be ran with the test as the first parameter.



### Step 3: Add tests

#### Testing cells are "Yes" or "No" (in list)

`in_list_test` generates a test that checks values in a column are in a given list

In [ ]:
%%writefile tests.py
from dftest import tests, declare_options

dftest_yes_or_no = tests.in_list_test(['yes', 'no', 'Yes', 'No'])
declare_options(dftest_yes_or_no, include=['B'])



Overwriting tests.py


#### Testing cells are balid IP addresses (match regex)

`match_test` generates test that checks values match a given regex.

In [ ]:
%%writefile -a tests.py
dftest_ip_addr = tests.match_test(r'([0-9]{1,3}\.){3}[0-9]{1,3}')
declare_options(dftest_ip_addr, include=['F'])



Appending to tests.py


#### Testing cells are not errors (nonequal)

`non_equal_test` generates tests that check values aren't equal to some value

In [ ]:
%%writefile -a tests.py
dftest_no_errors = tests.non_equal_test('ERROR')
declare_options(dftest_no_errors, include=['F'])



Appending to tests.py


#### Testing numerical cells are in range

`range_test` generates tests that check values are in a certain (bound or unbound) range. 

`is_fraction` is a range test that checks a value is between 0 and 1. `is_positive` checks values are a above 0. 

In [ ]:
%%writefile -a tests.py

from dftest.tests import dftest_fraction
dftest_percent = tests.in_range_test(0, 100, left_inclusive=True, right_inclusive=True)

declare_options(dftest_percent, include=['C', 'D'])
declare_options(tests.dftest_fraction, include=['E'])



Appending to tests.py


#### Type checking

`simple_type_test` generates a test that checks values are of a given type. `is_float`, `is_integer` and `is_str` are existing common derivations of it

In [ ]:
%%writefile -a tests.py

dftest_integer = tests.simple_type_test(int)

from dftest.tests import dftest_float, dftest_str

declare_options(dftest_integer, include=['C', 'D'])
declare_options(tests.dftest_float, include=['E'])
declare_options(tests.dftest_str, include=['B', 'F'])



Appending to tests.py


#### Null checking

`is_not_null` checks values are not `None` or `NaN`

In [ ]:
%%writefile -a tests.py
from dftest.tests import dftest_not_null

Appending to tests.py


### Step 4: Run!

In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: Type float — E)
Testing 06% (#2: In range [0, 1] — E)
Testing 11% (#3: Type int — C)
Testing 17% (#4: Type int — D)
Testing 22% (#5: Match /([0-9]{1,3}\.){3}[0-9]{1,3}/ — F)
Testing 28% (#6: Not ERROR — F)
Testing 33% (#7: dftest_not_null — A)
Testing 39% (#8: dftest_not_null — C)
Testing 44% (#9: dftest_not_null — E)
Testing 50% (#10: dftest_not_null — F)
Testing 56% (#11: dftest_not_null — D)
Testing 61% (#12: dftest_not_null — B)
Testing 67% (#13: dftest_not_null — Unnamed: 0)
Testing 72% (#14: In range [0, 100] — C)
Testing 78% (#15: In range [0, 100] — D)
Testing 83% (#16: Type str — F)
Testing 89% (#17: Type str — B)
Testing 94% (#18: In list ['yes', 'no', 'Yes', 'No'] — B)
Finished testing
Columns Tested: 7/7 (100%).
Columns valid: 2/7 (28.57%).

--- Column 3: B ---
Test #01: dftest_not_null — B: 6/6 (100.0%).

Test #02: Type str — B: 6/6 (100.0%).

Test #03: In list ['yes', 'no', 'Yes', 'No'] — B: 5/6 (83.33%).
   Unnamed: 0      B
4           4  Maybe

--- Col

## Demonstration 7: column autodetection

### Step 0: Setup - Composed IDs

Let's imagine a database which includes some column detailing years, some column detailing names and some ID column which should be the year number followed by a dash and the corresponding entry name.

In [ ]:
!pip3 install names

In [ ]:
import names, random

df = pd.DataFrame()
df['Year'] = [random.randint(2000, 2020) for i in range(100)]
df['Name'] = [names.get_full_name() for i in range(100)]
df.insert(0, 'ID', [f'{year}-{name.replace(" ", "")}' for year, name in zip(df['Year'], df['Name'])])
df.to_csv('example.csv', index=False)
pd.read_csv('example.csv')

,ID,Year,Name
0,2015-JohnCser,2015,John Cser
1,2010-ErnestWoods,2010,Ernest Woods
2,2008-DavidBernard,2008,David Bernard
3,2013-BrandyMassey,2013,Brandy Massey
4,2016-EarlCaldwell,2016,Earl Caldwell
...,...,...,...
95,2020-KevinHensley,2020,Kevin Hensley
96,2015-MargaritaWooley,2015,Margarita Wooley
97,2007-CarmenStallings,2007,Carmen Stallings
98,2002-JoyEliezrie,2002,Joy Eliezrie


### Step 1: Testing IDs

Let's say we'd like to test for malformed IDs.

In [ ]:
%%writefile tests.py
import numpy as np
from dftest import options

def dftest_id_form(dataframe):
  generated_ids =  dataframe['Year'].apply(str) + np.tile('-', 100) + dataframe['Name'].apply(lambda x: x.replace(' ', ''))
  return dataframe['ID'] == generated_ids

Overwriting tests.py


In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_id_form)
Finished testing
Columns Tested: 3/3 (100%).
Columns valid: 3/3 (100.0%).



### Step 2: Columns autodected

Putting in some invalid values:

In [ ]:
df.loc[:5, 'ID'] = 'invalid'
df.to_csv('example.csv', index=False)

In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_id_form)
Finished testing
Columns Tested: 3/3 (100%).
Columns valid: 0/3 (0.0%).

--- Column 1: ID ---
Test #01: dftest_id_form: 94/100 (94.0%).
        ID
0  invalid
1  invalid
2  invalid
3  invalid
4  invalid
5  invalid

--- Column 2: Year ---
Test #01: dftest_id_form: 94/100 (94.0%).
        ID  Year
0  invalid  2015
1  invalid  2010
2  invalid  2008
3  invalid  2013
4  invalid  2016
5  invalid  2011

--- Column 3: Name ---
Test #01: dftest_id_form: 94/100 (94.0%).
        ID              Name
0  invalid         John Cser
1  invalid      Ernest Woods
2  invalid     David Bernard
3  invalid     Brandy Massey
4  invalid     Earl Caldwell
5  invalid  Elizabeth Roemer



We can see the results for all three columns accessed. This is, in fact, one of the main features `dftest`; it can detect automatically which columns you are testing.

This feature generalizes to any columns accessed by `loc`, `iloc`, `__getitem__`, or attributes, via both DataFrame and Series. 

### Step 3: Overriding Autodetection

In the case dlineated above, we quite possibly don't actually think of our test as testing ID, Year and Name, but rather only testing ID, assuming Name and Year are valid.

In that case, we'll probably want to avoid the autodetction of Name and Year. We can do this by the `ignore_columns` and `columns_tested` options.


In [ ]:
%%writefile tests.py
import numpy as np
from dftest import options

@options(ignore_columns=['Year', 'Name']) # Specify columns autodetection should ignore
def dftest_id_form(dataframe):
  generated_ids =  dataframe['Year'].apply(str) + np.tile('-', 100) + dataframe['Name'].apply(lambda x: x.replace(' ', ''))
  return dataframe['ID'] == generated_ids

Overwriting tests.py


In [ ]:
%%writefile tests.py
import numpy as np
from dftest import options

@options(tested_columns=['ID']) # Specify columns autodetection should ignore
def dftest_id_form(dataframe):
  generated_ids =  dataframe['Year'].apply(str) + np.tile('-', 100) + dataframe['Name'].apply(lambda x: x.replace(' ', ''))
  return dataframe['ID'] == generated_ids

Overwriting tests.py


In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_id_form)
Finished testing
Columns Tested: 1/3 (33%).
Columns valid: 0/1 (0.0%).

--- Column 1: ID ---
Test #01: dftest_id_form: 94/100 (94.0%).
        ID
0  invalid
1  invalid
2  invalid
3  invalid
4  invalid
5  invalid



The `tested_columns` is only available for "concrete" tests that run over specific columns.

For genreic tests, we can turn off autodetection by specifying `column_autodetect=False`. In that case, each test will be assigned to the column it is applied to, only.

### Step 4: Edge cases

In some edge cases autodetection may not work properly; particularly when dealing with multiple dataframes.

Under the hood, the library tracks calls to `pandas.Index.__getitem__` which is called by all of the aforementioned methods.

however, there is no way to ensure that the calling object is the tested dataframe. Even if we compared the tested dataframe with the original caller, the test functions themselves may modify, add or slice the dataframe in ways that are completely valid, yet create a new object, all before actually acessing the column.

For this reason, if a column in a different dataframe is accessed, and that column has the same name as a column in the tested dataframe, that column of the tested dataframe will be autodetected.

As an example, let's say that the `Year` and `Name` coulumns are actually taken from some other dataframe, `some_origin_dataset.csv`, and that our actual tested dataframe, with the ID generated from that origin dataset, also has its own `Year` column, with Years 1900-1920.

In [ ]:
import names, random

df = pd.DataFrame()
df['Year'] = [random.randint(2000, 2020) for i in range(100)]
df['Name'] = [names.get_full_name() for i in range(100)]
df.to_csv('some_origin_dataset.csv', index=False)

df2 = pd.DataFrame()
df2['ID'] = [f'{year}-{name.replace(" ", "")}' for year, name in zip(df['Year'], df['Name'])]
df2['Year'] = [random.randint(1900, 1920) for i in range(100)]

df2.loc[:5, 'ID'] = 'invalid'
df2.to_csv('example.csv', index=False)

print(df)
print(df2)

    Year             Name
0   2001     Larry Cowart
1   2008   Bianca Mcguire
2   2008  Lottie Mcarthur
3   2005     Martin Smith
4   2019       Doris Reed
..   ...              ...
95  2011     Roger Jodoin
96  2018     Charles Cone
97  2009  Gregory Rodgers
98  2009      Boyd Rogers
99  2013      Dawn Dexter

[100 rows x 2 columns]
                     ID  Year
0               invalid  1918
1               invalid  1909
2               invalid  1915
3               invalid  1918
4               invalid  1900
..                  ...   ...
95     2011-RogerJodoin  1911
96     2018-CharlesCone  1902
97  2009-GregoryRodgers  1904
98      2009-BoydRogers  1916
99      2013-DawnDexter  1902

[100 rows x 2 columns]


Farthermore, let's remove the options decorator, letting the columns be autododetected.

In [ ]:
%%writefile tests.py
import numpy as np
import pandas as pd
from dftest import options

def dftest_id_form(dataframe):
  origin_df = pd.read_csv('some_origin_dataset.csv')
  generated_ids =  origin_df['Year'].apply(str) + np.tile('-', 100) + origin_df['Name'].apply(lambda x: x.replace(' ', ''))
  return dataframe['ID'] == generated_ids

Overwriting tests.py


In [ ]:
!dftest --dataframe example.csv --files tests.py

Testing 00% (#1: dftest_id_form)
Finished testing
Columns Tested: 2/2 (100%).
Columns valid: 0/2 (0.0%).

--- Column 1: ID ---
Test #01: dftest_id_form: 94/100 (94.0%).
        ID
0  invalid
1  invalid
2  invalid
3  invalid
4  invalid
5  invalid

--- Column 2: Year ---
Test #01: dftest_id_form: 94/100 (94.0%).
        ID  Year
0  invalid  1918
1  invalid  1909
2  invalid  1915
3  invalid  1918
4  invalid  1900
5  invalid  1912



In that case, we can see that the `Year` column in the tested dataframe, which we in fact did not do anything with, was detected as a tested column. What happened was that the call `origin_df['Year']` was detected, and `'Year'` was determined as a valid column name from the tested dataframe; thus `dftest` registered it as a tested column.

A relatively simple way to avoid this is to just specify the `ignore_columns` options, but this behaviour is important to keep in mind.

## Demonstration 8: divergent tests

### Step 0: Divergent Tests?

Both the type of the test result (Index vs. Boolean) and the columns that were tested (see Demonstration 7) are determined at runtime. This means we can a make a test that conditionally returns more information, and that if a test conditionally accesses columns, the columns that will show up as tested and covered would only be the ones it *actually* accessed at runtime.

### Step 1: A result-divergent test

As an example, we may want to get more detailed error report, only if there are errors beyond a certain threshold, and otherwise we'd to not waste processing time and just return a valid/invalid binary.

the following test checks a sample of 300 rows for cells outside the unit interval, and only if it finds more than 15 it will see it important enough to tell exactly which lines; otherwise it will just mark the test succesful. 

In [ ]:
%%writefile tests.py

def dftest_unit_interval(col, df):
  sample_errors = sum(1 for cell in df.sample(300)[col] if not 0 <= cell <= 1)
  if sample_errors >= 15:
    return df[col].apply(lambda x: 0 <= x <= 1)
  return True

Overwriting tests.py


Running it on an example dataset

In [ ]:
df =  pd.DataFrame(np.random.randint(0, 100, size=(1000, 4))/100, columns=list('ABCD'))
df['B'][:200] *= 120
df['C'][:10] *= 120

df.to_csv('example.csv', index=False)
df

,A,B,C,D
0,0.90,8.40,104.40,0.75
1,0.56,22.80,69.60,0.82
2,0.21,28.80,102.00,0.26
3,0.45,16.80,44.40,0.34
4,0.41,92.40,34.80,0.55
...,...,...,...,...
995,0.92,0.57,0.31,0.06
996,0.77,0.64,0.51,0.85
997,0.37,0.91,0.76,0.59
998,0.25,0.95,0.97,0.99


In [ ]:
!dftest --dataframe example.csv --files tests.py --column B
!dftest --dataframe example.csv --files tests.py --column C

Testing 00% (#1: dftest_unit_interval — B)
Testing 25% (#2: dftest_unit_interval — C)
Testing 50% (#3: dftest_unit_interval — A)
Testing 75% (#4: dftest_unit_interval — D)
Finished testing
--- B ---
Test #01: dftest_unit_interval — B: 801/1000 (80.1%).
      B
0   8.4
1  22.8
2  28.8
3  16.8
4  92.4
5  76.8
6  61.2
7  21.6
8  37.2
9  20.4
...

Testing 00% (#1: dftest_unit_interval — A)
Testing 25% (#2: dftest_unit_interval — C)
Testing 50% (#3: dftest_unit_interval — B)
Testing 75% (#4: dftest_unit_interval — D)
Finished testing
--- C ---
Test #01: dftest_unit_interval — C: Success



### Step 2: A column-divergent test

As a second example, let's say we have a dataset where, by an old standard, an item's ID was a hash of the `fullname` column, but since 2015 that standard was changed such that an ID is a `yymmdd` formatting of the `CreationTimestamp` column. We want to test this behavior

Our function may look something like this:

In [ ]:
%%writefile tests.py
from datetime import datetime

def dftest_id(dataframe):
  creation_times =  dataframe['CreationTimestamp'].apply(datetime.fromtimestamp)
  return \
    creation_times >= 2018 and dataframe['ID'] == creation_times.apply(lambda x: x.strftime('%y%m%d')) \
    or creation_times < 2018 and dataframe['ID'] == dataframe['fullname'].apply(hash)

Notice that due to the short circuiting nature of python `fullname` would only be accessed if there are creation times.

On a normal run, all `fullname`, `CreationTimestamp` and `ID` would all be autodected. However, imagine we have a different dataframe for each year. In that case, we would see a loss of coverage immediately following the standard change, as `fullname` would no longer be accessed and thus detected.

## Demonstration 11: styling output graphs 

You can set the colorscheme graphs use to communicate validity using a JSON file.

A color scheme is just a list of color-number pairs, with numbers starting from 0 and ending in 1. The corresponding colors for 0 and 1 are used for binary invalid/valid data. Continous data may be classified (in the tests success grah for individual columns) to the color with the lower smaller or equal value, or mapped to a gradient generated from the list.

Thus, the default style looks like so:
```json
[
  {"red": 0},
  {"orange": 0.25}
  {"yellow": 0.5}
  {"blue": 0.75}
  {"green": 1}
]
```

colors don't have to be equally spaced.

Colorschemes like shown above should be paired with a column name, so 
```json
  "Object Number": [
    {"purple": 0},
    {"red": 0.5},
    {"yellow": 1}
  ]
```
will make graphs for the "Object Number" column display in the specified colorscheme.

You can use the special name `__DEFAULT__` to assign default color schemes, and the special name `__DATAFRAME__` to assign colorschemes to dataframe graphs.

The following style file will color dataframe graphs in monochrome, switch orange and blue for all columns but A, and reduce A to an uneven red-green-blue scheme:

In [ ]:
%%writefile style.json
{
  "__DATAFRAME__": [
    {"white": 0},
    {"lightgray": 0.25},
    {"silver": 0.5},
    {"darkgray": 0.75},
    {"black": 1}
  ],

  "__DEFAULT__": [
    {"red": 0},
    {"blue": 0.25},
    {"yellow": 0.5},
    {"orange": 0.75},
    {"green": 1}
  ],

  "A": [
    {"red": 0},
    {"green": 0.75},
    {"blue": 1}
  ]
}

You can specify a style file to use with the `--style` option.